In [7]:
# pip install --upgrade yfinance
import yfinance as yf
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsedata.bse import BSE

In [8]:
# Read the CSV file
df = pd.read_csv('CsvFiles/trends.csv')

# Extract the company codes
bse_codes = df['BSE Code'].dropna().tolist()  # Drop missing values and convert to list
nse_codes = df['NSE Code'].dropna().tolist()  # Drop missing values and convert to list

In [9]:
len(nse_codes)

1013

In [10]:
%%capture
# Define a function to calculate moving averages and percentages
def calculate_moving_averages_and_percentages(symbol):
    # Fetch stock data
    data = yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365), end=pd.to_datetime('today') + pd.DateOffset(1), progress=False, auto_adjust=False)
    print(data)
    data.columns = [col[0] for col in data.columns]
    #data = yf.download(symbol + ".NS", period="1y", actions=True)
        
    if data.empty:
        return None  # Skip if no data is fetched
    
    # Calculate 20DMA, 50DMA, and 200DMA
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['200DMA'] = data['Close'].rolling(window=200).mean()
    
    # Calculate percentages of days where Close > DMA
    total_days = len(data)
    if total_days == 0:
        return None
    
    percent_20DMA = ((data['Close'] > data['20DMA']).sum() / total_days) * 100
    percent_50DMA = ((data['Close'] > data['50DMA']).sum() / total_days) * 100
    percent_200DMA = ((data['Close'] > data['200DMA']).sum() / total_days) * 100
    
    return {
        "Stock": symbol,
        "Percent_20DMA": percent_20DMA,
        "Percent_50DMA": percent_50DMA,
        "Percent_200DMA": percent_200DMA
    }
        

# Process each stock
results = []
for symbol in nse_codes:
    result = calculate_moving_averages_and_percentages(symbol)
    if result:
        results.append(result)

# Create a summary DataFrame
summary_df = pd.DataFrame(results)
summary_df = summary_df.sort_values(by='Percent_20DMA', ascending=False)  # Sort by 50DMA percentage

KeyError: 'Percent_20DMA'

In [5]:
summary_df.to_csv('CsvFiles/trends_results.csv', index=False)

In [6]:
def download_stock_data(symbol):
    data = yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365), end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
    data.columns = [col[0] for col in data.columns]
    return data
    
    
def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data
    

def plot_stock_data(symbol, data):
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=200))]
    
    # Setup subplot layout
    figure = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    
    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'), row=1, col=1)
    
    # Plot moving averages
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'), row=1, col=1)

    # Plot volume with color-coded bars (green for positive, red for negative days)
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True, row=1, col=1)

    # Update axes
    figure.update_yaxes(range=[plot_data['Close'].min()*0.8, plot_data['Close'].max()*1.2], row=1, col=1)
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # Hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True, row=1, col=1)
    figure.update_yaxes(visible=False, secondary_y=True, row=1, col=1)

    return figure


def process_stocks(all_symbols, page):
    figure_html = open(f'Plots/Trend{page}.html', 'w')
    for symbol in all_symbols:
        try:
            data = download_stock_data(symbol)
            data = calculate_technical_indicators(data)
            figure = plot_stock_data(symbol, data)
            figure_html.write(figure.to_html(full_html=False))
        except Exception as e:
            e 
    figure_html.close()


if __name__ == "__main__":
    symbols = summary_df.sort_values('Percent_20DMA', ascending=False)['Stock'][:100].tolist()
    process_stocks(symbols, 1)
    # symbols = summary_df.sort_values('Percent_50DMA', ascending=False)['Stock'][100:200].tolist()
    # process_stocks(symbols, 2)
    print('Execution Completed!!!!')

KeyError: 'Percent_20DMA'